In [5]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd

# criar o navegador
servico = Service(ChromeDriverManager().install())
nav = webdriver.Chrome(service=servico)

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,1000,3500
1,rtx 3060,zota galax,1000,4500


Definição das funções de busca no google e no buscape

In [6]:
import time

def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos
    
def verificar_tem_todos_termos_produtos(lista_termos_produto, nome):
    tem_todos_termos_produto = True
    for palavra in lista_termos_produto:
        if palavra not in nome:
            tem_todos_termos_produto = False
    return tem_todos_termos_produto

    

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # entrar no google
    nav.get("https://www.google.com/")
    
    # tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    

    # pesquisar o nome do produto no google
    nav.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto)
    nav.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(Keys.ENTER)
    
    time.sleep(2)
    
    # clicar na aba shopping
    elementos = nav.find_elements(By.CLASS_NAME, 'YmvwI')
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # pegar a lista de resultados da busca no google shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'KZmu8e')
    
    time.sleep(2)
    
    # para cada resultado, ele vai verificar se o resultado corresponde a todas as nossas condicoes
    lista_ofertas = [] # lista que a função vai me dar como resposta
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'sh-np__product-title').text
        nome = nome.lower()
        
        # verificacao do nome - se no nome tem algum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        
        # verificar se no nome tem todos os termos do nome do produto
        tem_todos_termos_produto = verificar_tem_todos_termos_produtos(lista_termos_produto, nome)

        if not tem_termos_banidos and tem_todos_termos_produto: # verificando o nome
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'hn9kf').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                # verificando se o preco ta dentro do minimo e maximo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'ROMz4c')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue

            
    return lista_ofertas
    
    

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    
    # entrar no buscape
    nav.get("https://www.buscape.com.br/")
    
    time.sleep(2)
    # pesquisar pelo produto no buscape
    nav.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__WAC2Y').send_keys(produto, Keys.ENTER)
    
    # pegar a lista de resultados da busca do buscape
    while len(nav.find_elements(By.CLASS_NAME, 'Select_Select__1HNob')) < 1:
        time.sleep(1)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh')
    
    # para cada resultado
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__HEz7L').text
            nome = resultado.find_element(By.CLASS_NAME, 'ProductCard_ProductCard_Name__U_mUQ').text
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            # verificacao do nome - se no nome tem algum termo banido
            tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome) 
                    
            # verificar se no nome tem todos os termos do nome do produto
            tem_todos_termos_produto = verificar_tem_todos_termos_produtos(lista_termos_produto, nome)
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
    return lista_ofertas

Contrução da nossa lista de ofertas encontradas

In [7]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None

display(tabela_ofertas)

,produto,preco,link
0,"apple iphone 12 64gb preto 5g tela 6,1 câm. tr...",3099.00,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...
1,"iphone 12 apple 64gb preto tela 6,1 12mp ios, ...",2789.10,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...
2,"iphone 12 apple 64gb branco tela 6,1 12mp ios,...",2789.10,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...
3,iphone 12 64gb azul outlet,1529.15,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...
4,iphone 12 preto apple 64gb 5g camera dupla 12m...,2340.00,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...
5,iphone 12 64gb preto outlet,1529.15,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...
6,apple iphone 12 branco 5g 64gb camera 12mpx vi...,2195.00,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...
7,"iphone 12 apple 64gb verde tela 6,1 12mp ios, ...",2789.10,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...
8,iphone 12 branco apple 64gb 5g camera dupla 12...,2059.00,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...
9,"iphone 12 64gb roxo tela 6,1 4g câmera traseir...",2199.00,https://www.google.com/aclk?sa=l&ai=DChcSEwiU1...


Exportar a base de ofertas para Excel

In [8]:
# exportar pro excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)


Enviando o e-mail

In [9]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'pedrofernandes20014@hotmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()

nav.quit()  